In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
import numpy as np
import statistics

from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.inspection import permutation_importance

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/AlexanderOG/AI_CRI/main/train.csv')
test_data = pd.read_csv("https://raw.githubusercontent.com/AlexanderOG/AI_CRI/main/test.csv")

## **1. Feature Engineering**

In [3]:
#Encoding the Sex variable
def encode_sex(x):
    if x == 'male':
        return 0
    elif x == 'female':
        return 1

data['Sexcode'] = data['Sex'].apply(encode_sex)
test_data['Sexcode'] = test_data['Sex'].apply(encode_sex)

In [4]:
#Creating a new variable: 5 categories for the different prices paid (Fare)
def categorize_fare(x):
  if x == 0:
        return 0
  elif x < 15:
        return 1
  elif x < 30:
        return 2
  elif x < 100:
        return 3
  else:
        return 4
data['FareCat']= data['Fare'].apply(categorize_fare)
test_data['FareCat']= test_data['Fare'].apply(categorize_fare)

In [5]:
#Creating a new variable with the number of family members each passenger has on board
data['FamilyCount']=data['Parch']+data['SibSp']+1
test_data['FamilyCount']=test_data['Parch']+test_data['SibSp']+1

In [6]:
#Filling missing values in Age with the median
data['Age']= data['Age'].fillna(statistics.median(data['Age'].dropna()))
test_data['Age']= test_data['Age'].fillna(statistics.median(test_data['Age'].dropna()))
#Filling missing values in Fare
test_data['Fare']= test_data['Fare'].fillna(statistics.median(test_data['Fare'].dropna()))

In [7]:
#Creating 5 categories for Age to have values ranging from 0 to 4
def categorize_age(x):
    if x <= 16:
        return 0
    elif x <= 32:
        return 1
    elif x <= 48:
        return 2
    elif x <= 64:
        return 3
    elif x <= 80:
        return 4

data['AgeCat']=data['Age'].apply(categorize_age)
test_data['AgeCat']= test_data['Age'].apply(categorize_age)

In [8]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [9]:
#Replacing missing values with most frequent one
data['Embarked'] = data['Embarked'].fillna('S')
test_data['Embarked'] = test_data['Embarked'].fillna('S')

In [10]:
data['Embarked_code'] = data['Embarked'].map( {'Q': 0, 'C': 1, 'S': 2} )
test_data['Embarked_code'] = test_data['Embarked'].map( {'Q': 0, 'C': 1, 'S': 2} )

## **2. KNN model**

In [11]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sexcode,FareCat,FamilyCount,AgeCat,Embarked_code
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1,2,1,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,3,2,2,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,1,1,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,3,2,2,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,1,1,1,2
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,3,1,1,2
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,NaN,S,1,2,4,1,2
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0,3,1,1,1


In [12]:
X = data.drop(columns = ['Survived', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age', "PassengerId"])
y = data.Survived
X

,Pclass,SibSp,Parch,Fare,Sexcode,FareCat,FamilyCount,AgeCat,Embarked_code
0,3,1,0,7.2500,0,1,2,1,2
1,1,1,0,71.2833,1,3,2,2,1
2,3,0,0,7.9250,1,1,1,1,2
3,1,1,0,53.1000,1,3,2,2,2
4,3,0,0,8.0500,0,1,1,2,2
...,...,...,...,...,...,...,...,...,...
886,2,0,0,13.0000,0,1,1,1,2
887,1,0,0,30.0000,1,3,1,1,2
888,3,1,2,23.4500,1,2,4,1,2
889,1,0,0,30.0000,0,3,1,1,1


In [13]:
# Instanciate the model
knn_model = KNeighborsRegressor()
knn_model.fit(X_train, y_train);

# Train the model on the scaled Training data
cv_results = cross_validate(knn_model, X,y)

base_knn_score = cv_results['test_score'].mean()

base_knn_score

NameError: name 'X_train' is not defined

In [ ]:
scaler = MinMaxScaler()

X_rescaled = scaler.fit_transform(X)

cv_results = cross_validate(knn_model, X_rescaled,y)

rescaled_score = cv_results['test_score'].mean()

rescaled_score

In [ ]:
score = []
neighbours = []

for k in range(1,25):
    
    # Instanciate the model
    knn_model = KNeighborsRegressor(n_neighbors = k)

    # Train the model on the scaled Training data
    cv_results = cross_validate(knn_model, X_rescaled,y)

    # Append the score and k
    score.append(cv_results['test_score'].mean())
    neighbours.append(k)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,25),score,color='blue', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.title('Score vs. $k$ Neighbors')
plt.xlabel('$k$')
plt.ylabel('R2 score')

In [ ]:
# The best k seems to be 24
best_k = 5

In [ ]:
# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
train_sizes, train_scores, test_scores = learning_curve(estimator = KNeighborsRegressor(n_neighbors=best_k),
                                                              X = X_rescaled, 
                                                              y = y, 
                                                              train_sizes = [50,100,250,500,750], 
                                                              cv = 10,
                                                              scoring = 'r2')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

# Plot the learning curves!
plt.plot(train_sizes, train_scores_mean, label = 'Training score')
plt.plot(train_sizes, test_scores_mean, label = 'Test score')
plt.ylabel('r2 score', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves', fontsize = 18, y = 1.03)
plt.legend()

In [ ]:
train_scores_mean

In [ ]:
cv_results = cross_validate(KNeighborsRegressor(n_neighbors=best_k), X_rescaled,y, scoring = 'neg_mean_absolute_error')

cat_error = cv_results['test_score'].mean()
cat_error

# **3. LDA model**

In [ ]:
X = data[["Pclass","SibSp","Parch","Fare","Sexcode","FareCat","FamilyCount","AgeCat","Embarked_code"]]
y = data.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
lda_model = LDA()
lda_model.fit(X_train, y_train);

In [ ]:
# Prediction on the training set to see model accuracy
y_predicted = lda_model.predict(X_train)
fails = sum((y_train - y_predicted)!=0)
corrects = sum((y_train - y_predicted)==0)
acc = 1 - fails/corrects
print("Accuracy with the training set:",acc)
print("R2 score with the training set:",r2_score(y_train, y_predicted))

# Prediction on the test set to see model accuracy
y_predicted = lda_model.predict(X_test)
fails = sum((y_test - y_predicted)!=0)
corrects = sum((y_test - y_predicted)==0)
acc = 1 - fails/corrects
print("Accuracy with the test set:",acc)
print("R2 score with the test set:",r2_score(y_test, y_predicted))

In [ ]:
# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
scaler = MinMaxScaler()
X_rescaled = scaler.fit_transform(X)

train_sizes, train_scores, test_scores = learning_curve(estimator = lda_model,
                                                              X = X_rescaled, 
                                                              y = y, 
                                                              train_sizes = [50,100,250,500,750], 
                                                              cv = 10,
                                                              scoring = 'r2')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

# Plot the learning curves!
plt.plot(train_sizes, train_scores_mean, label = 'Training score')
plt.plot(train_sizes, test_scores_mean, label = 'Test score')
plt.ylabel('r2 score', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves with all variables', fontsize = 18, y = 1.03)
plt.legend()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 15)
plt.tight_layout(pad = 1) # adjust the subplots to fit in to the figure area.
plt.subplots_adjust(hspace = 0.5, wspace = 0.3) # adjust the amount of width and height reserved for space between subplots.

all_variables = ["Pclass","SibSp","Parch","Fare","Sexcode","FareCat","FamilyCount","AgeCat","Embarked_code"]
y = data.Survived
for i in range(len(all_variables)):
  without = all_variables[i]
  using = all_variables.copy()
  using.remove(without)

  X = data[using]
  scaler = MinMaxScaler()
  X_rescaled = scaler.fit_transform(X)
  train_sizes, train_scores, test_scores = learning_curve(estimator = lda_model,
                                                                X = X_rescaled, 
                                                                y = y, 
                                                                train_sizes = [50,100,250,500,750], 
                                                                cv = 10,
                                                                scoring = 'r2')

  # Take the mean of cross-validated train scores and validation scores
  train_scores_mean = np.mean(train_scores, axis=1)
  test_scores_mean = np.mean(test_scores, axis=1)

  # Plot the learning curves!
  plt.subplot(3, 3, i+1)
  plt.plot(train_sizes, train_scores_mean, label = 'Training score')
  plt.plot(train_sizes, test_scores_mean, label = 'Test score')
  plt.ylabel('r2 score')
  plt.xlabel('Training set size')
  plt.title('Learning curves without '+ without,  y = 1.03)
  plt.legend()

plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
result = permutation_importance(lda_model, X_test, lda_model.predict(X_test), n_repeats=10, random_state=0)
result

In [ ]:
features = ["Pclass","SibSp","Parch","Fare","Sexcode","FareCat","FamilyCount","AgeCat","Embarked_code"]

In [ ]:
indices = np.argsort(result.importances_mean)[::-1]
indices
ordered_features = []
for i in indices:
  ordered_features.append(features[i])

In [ ]:
result.importances_mean[indices]

In [ ]:
result.importances_std[indices]

In [ ]:
plt.figure()
plt.title("Feature importance")
plt.bar(ordered_features, result.importances_mean[indices],
        color="r", yerr=result.importances_std[indices], align="center")
#plt.xticks(range(X.shape[1]), indices)
#plt.xlim([-1, X.shape[1]])
plt.show()

# **4. Logistic Regression**

In [ ]:
X = data[["Pclass","SibSp","Parch","Fare","Sexcode","FareCat","FamilyCount","AgeCat","Embarked_code"]]
y = data.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
reg_model = LogisticRegression(random_state=0, max_iter=4000)
reg_model.fit(X_train, y_train)

In [ ]:
reg_model.score(X_train, y_train)

In [ ]:
# Prediction on the training set to see model accuracy
y_predicted = reg_model.predict(X_train)
fails = sum((y_train - y_predicted)!=0)
corrects = sum((y_train - y_predicted)==0)
acc = 1 - fails/corrects
print("Accuracy with the training set:",acc)
print("R2 score with the training set:",r2_score(y_train, y_predicted))

# Prediction on the test set to see model accuracy
y_predicted = reg_model.predict(X_test)
fails = sum((y_test - y_predicted)!=0)
corrects = sum((y_test - y_predicted)==0)
acc = 1 - fails/corrects
print("Accuracy with the test set:",acc)
print("R2 score with the test set:",r2_score(y_test, y_predicted))

In [ ]:
# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
scaler = MinMaxScaler()
X_rescaled = scaler.fit_transform(X)

train_sizes, train_scores, test_scores = learning_curve(estimator = reg_model,
                                                              X = X_rescaled, 
                                                              y = y, 
                                                              train_sizes = [50,100,250,500,750], 
                                                              cv = 10,
                                                              scoring = 'r2')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

# Plot the learning curves!
plt.plot(train_sizes, train_scores_mean, label = 'Training score')
plt.plot(train_sizes, test_scores_mean, label = 'Test score')
plt.ylabel('r2 score', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves with all variables', fontsize = 18, y = 1.03)
plt.legend()

### **Model comparison**

In [ ]:
X_train = data[["Pclass","SibSp","Parch","Fare","Sexcode","FareCat","FamilyCount","AgeCat","Embarked_code"]]
y_train = data.Survived
knn_model.fit(X_train, y_train);
lda_model.fit(X_train, y_train);
reg_model.fit(X_train, y_train); 

In [ ]:
kaggle_test = test_data[["Pclass","SibSp","Parch","Fare","Sexcode","FareCat","FamilyCount","AgeCat","Embarked_code"]]
y_pred_KNN = knn_model.predict(kaggle_test)
y_pred_LDA = lda_model.predict(kaggle_test)
y_pred_REG = reg_model.predict(kaggle_test)
#is_NaN = kaggle_test.isnull()
#row_has_NaN = is_NaN.any(axis=1)
#rows_with_NaN = kaggle_test[row_has_NaN]
#rows_with_NaN

In [ ]:
#print("R² of KNN: ", r2_score(y_test, y_pred_KNN))
#print("R² of LDA: ", r2_score(y_test, y_pred_LDA))
#print("R² of REG: ", r2_score(y_test, y_pred_LDA))

In [ ]:
class_y_pred_KNN = [round(x) for x in y_pred_KNN]

In [ ]:
y_pred_REG_Survived = pd.Series(y_pred_REG)
y_pred_LDA_Survived = pd.Series(y_pred_LDA)
y_pred_KNN_Survived = pd.Series(class_y_pred_KNN)
pred_Id = test_data["PassengerId"]

prediction_df_KNN = pd.DataFrame({"PassengerId":pred_Id,"Survived":class_y_pred_KNN})
prediction_df_LDA = pd.DataFrame({"PassengerId":pred_Id,"Survived":y_pred_LDA_Survived})
prediction_df_REG = pd.DataFrame({"PassengerId":pred_Id,"Survived":y_pred_REG_Survived})
prediction_df_KNN#.to_csv("knn_prediction.csv",index=False)
prediction_df_LDA#.to_csv("lda_prediction.csv",index=False)
prediction_df_REG#.to_csv("reg_prediction.csv",index=False)